# Extracting Data From the WMO app

## 1 Insturactions of the WMO

Use the following steps to carry out the project
- Use this link (https://community.wmo.int/en/members/profiles to extract data on
heatwaves warning system/services, see example below, if heatwave is in the list of
warnings provided, then indicate a yes in the provided excel sheet (for the corresponding
country in column B in the excel sheet called “Heatwaves Warning Systems (WMO data)”).


full screenmode : https://app.powerbi.com/view?r=eyJrIjoiZjNhNzIzM2YtMjRkYS00ZjJjLWEzZmMtNmQzMGQzMDdiODU3IiwidCI6ImVhYTZiZTU0LTQ2ODctNDBjNC05ODI3LWMwNDRiZDhlOGQzYyIsImMiOjl9


## 2  Required Libraries

In [137]:
#The webdriver module in Selenium allows for automated control of web browsers.


from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC




## 3 Opening up the web browser and printing a sample text

The function below opens up chrome browser and waits for 60 minutes to load the data 

input: The website link

Output : the driver and wabdriverwait instance

In [138]:

def openChrome(link):
    # Options customize Chrome WebDriver behavior
    options = Options()

    # Initialize the WebDriver (e.g., Chrome)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    # Navigate to the desired URL
    driver.get(link)

    # wait for page to load
    wait = WebDriverWait(driver, 60)

    return driver, wait

In [139]:
link = "https://app.powerbi.com/view?r=eyJrIjoiZjNhNzIzM2YtMjRkYS00ZjJjLWEzZmMtNmQzMGQzMDdiODU3IiwidCI6ImVhYTZiZTU0LTQ2ODctNDBjNC05ODI3LWMwNDRiZDhlOGQzYyIsImMiOjl9"
driver_instance, wait  = openChrome(link)

Using the driver instance that we created print an sample text

In [140]:
# Locate all div elements with class "textbox"
div_elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="textbox"]')))

# Iterate over the list of elements and print the text content of each
for div_element in div_elements:
    print(div_element.text)
    print("-----")  # Just a separator for clarity

# driver_instance.quit()

Member Profiles
-----
The information contained in these profiles have been provided by WMO Members. The information is based on a self-assessment exercise. The date displayed in each section corresponds with the date in which the information was provided to WMO. 

If you find any issue or inconsistency with the information displayed, please contact community@wmo.int.
-----
Permanent Representative:
-----
Hydrological Adviser:
-----
National Meteorological Service:
-----
Responsible Ministry:
-----


## 4. Clicking on the Early Warning Services 

In [141]:
#EC stands for "Expected Conditions." It is a set of predefined conditions that the Selenium WebDriver uses.
#presence_of_element_located is one of many expected conditions (EC). It checks if an element is present on the DOM of a page.
def wait_and_click(driver, wait, css_selector):
    element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))
    driver.execute_script("arguments[0].click();", element)

In [142]:
css_selector = 'a[title=" Early Warning Services"]'
wait_and_click(driver_instance, wait, css_selector)

## 5 - Get Warnings List

In [143]:
def saveWarnings(driver, css_selector):
    #'.title' is the class name of the element that contains the warning text
    warnings= driver_instance.find_elements(By.CSS_SELECTOR, css_selector)
    return warnings

warnings = saveWarnings(driver_instance, '.title')


In [144]:

for warning in warnings:
    print(warning.text)

## 6 selecting the next country

In [145]:

dropdown = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '.slicer-dropdown-menu'))
)
dropdown.click()

# Wait for the dropdown options to be visible
dropdown_options = wait.until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '[role="option"]'))  # Assuming each dropdown option has a role of "option"
)

country_names = []
for option in dropdown_options:
    country_name = option.text
    country_names.append(country_name)

    # Uncomment the following lines if you want to click on a specific country
   
    if country_name == "Angola":  
        option.click()
        break
    

print(country_names)

['', '', '', '', '', '', '', '', '', '', '', '', 'Ground movement', 'Riverine flood', 'Flash flood', 'General flood', '', '', '', '', 'Ground moveme…', 'Riverine flood', 'General flood', 'Flash flood', 'Heat wave', 'Forest fire', 'General storm', 'General landslide', 'Coastal flood', 'Cold wave', 'Drought', 'Locust', '', '', '', '', '', '', '', '', '', '', '', '']


## 6 select the next country

from a manual country name entery, switch the tab, print the list 

In [155]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

options = Options()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Navigate to the desired URL
driver.get('https://app.powerbi.com/view?r=eyJrIjoiZjNhNzIzM2YtMjRkYS00ZjJjLWEzZmMtNmQzMGQzMDdiODU3IiwidCI6ImVhYTZiZTU0LTQ2ODctNDBjNC05ODI3LWMwNDRiZDhlOGQzYyIsImMiOjl9')

# Open the dropdown menu
dropdown = WebDriverWait(driver, 60).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '.slicer-dropdown-menu'))
)
dropdown.click()

loaded_countries = set()

while True:
    # Re-fetch the dropdown options in every iteration to avoid StaleElementReferenceException
    current_countries_elements = WebDriverWait(driver, 60).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.slicerText'))
    )
    
    # Extract country names from the current elements
    current_countries = {element.text for element in current_countries_elements}

    # Check if no new countries are found
    if not current_countries - loaded_countries:
        break
    
    loaded_countries.update(current_countries)

    # Scroll to the last element to load more countries
    ActionChains(driver).move_to_element(current_countries_elements[-1]).perform()

print(loaded_countries)


{'Austria', 'Poland', 'Barbados', 'Burundi', 'Papua New Guinea', 'Lebanon', 'Yemen', 'Central African Republic', 'Ireland', 'Iceland', 'Finland', 'Ecuador', 'French Polynesia', 'Israel', 'Kuwait', 'Nigeria', 'North Macedonia', 'Albania', 'Colombia', 'Malta', 'Japan', 'Cook Islands', 'Spain', 'Venezuela, Bolivarian Republic of', 'Curaçao and Sint Maarten', 'Lesotho', 'Mali', 'Seychelles', 'Iraq', 'Bhutan', 'Hungary', 'Czechia', 'Suriname', 'Panama', 'Haiti', 'Kenya', 'Paraguay', 'British Caribbean Territories - Montserrat', 'Jordan', 'British Caribbean Territories - Turks and Caicos Islands', 'Andorra', 'Mauritania', 'Niger', 'Türkiye', 'Germany', 'Chad', 'Qatar', 'Trinidad and Tobago', 'Nauru', 'Vanuatu', 'British Caribbean Territories - Cayman Islands', 'Republic of Korea', 'Monaco', 'Peru', 'Bosnia and Herzegovina', 'Ghana', 'United States of America', 'Botswana', 'Mexico', 'New Zealand', 'Bangladesh', 'Myanmar', 'Samoa', 'Malaysia', 'Australia', 'Tonga', 'Cambodia', 'Honduras', 'Bol

In [156]:
loaded_countries

{'Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia (Plurinational State of)',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'British Caribbean Territories',
 'British Caribbean Territories - Anguilla',
 'British Caribbean Territories - British Virgin Islands',
 'British Caribbean Territories - Cayman Islands',
 'British Caribbean Territories - Montserrat',
 'British Caribbean Territories - Turks and Caicos Islands',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Cook Islands',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Curaçao and Sint Maarten',
 'Cyprus',
 'Czechia',
 "Côte d'Ivoire",
 "Democratic People's

In [157]:
len(loaded_countries)

198

stop when you see Spain

In [153]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

options = Options()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Navigate to the desired URL
driver.get('https://app.powerbi.com/view?r=eyJrIjoiZjNhNzIzM2YtMjRkYS00ZjJjLWEzZmMtNmQzMGQzMDdiODU3IiwidCI6ImVhYTZiZTU0LTQ2ODctNDBjNC05ODI3LWMwNDRiZDhlOGQzYyIsImMiOjl9')

# Open the dropdown menu
dropdown = WebDriverWait(driver, 60).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '.slicer-dropdown-menu'))
)
dropdown.click()

loaded_countries = set()
target_country = "Spain"
country_selected = False

while True:
    # Re-fetch the dropdown options in every iteration to avoid StaleElementReferenceException
    current_countries_elements = WebDriverWait(driver, 60).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.slicerText'))
    )
    
    # Extract country names from the current elements
    current_countries = {element.text for element in current_countries_elements}

    # Check if target country is in the current batch of loaded countries
    if target_country in current_countries:
        for option in current_countries_elements:
            if option.text == target_country:
                option.click()
                country_selected = True
                break

    # If the country has been selected, break out of the loop
    if country_selected:
        break
    
    # Check if no new countries are found
    if not current_countries - loaded_countries:
        break
    
    loaded_countries.update(current_countries)

    # Scroll to the last element to load more countries
    ActionChains(driver).move_to_element(current_countries_elements[-1]).perform()

print(loaded_countries)


{'Austria', 'Poland', 'Barbados', 'Burundi', 'Papua New Guinea', 'Lebanon', 'Central African Republic', 'Ireland', 'Iceland', 'Finland', 'Ecuador', 'French Polynesia', 'Israel', 'Kuwait', 'Nigeria', 'North Macedonia', 'Albania', 'Colombia', 'Malta', 'Japan', 'Cook Islands', 'Curaçao and Sint Maarten', 'Lesotho', 'Mali', 'Seychelles', 'Iraq', 'Bhutan', 'Hungary', 'Czechia', 'Panama', 'Haiti', 'Kenya', 'Paraguay', 'British Caribbean Territories - Montserrat', 'Jordan', 'British Caribbean Territories - Turks and Caicos Islands', 'Andorra', 'Mauritania', 'Niger', 'Germany', 'Chad', 'Qatar', 'Nauru', 'British Caribbean Territories - Cayman Islands', 'Republic of Korea', 'Monaco', 'Peru', 'Bosnia and Herzegovina', 'Ghana', 'Botswana', 'Mexico', 'New Zealand', 'Bangladesh', 'Myanmar', 'Samoa', 'Malaysia', 'Australia', 'Cambodia', 'Honduras', 'Bolivia (Plurinational State of)', 'New Caledonia', 'Luxembourg', 'Afghanistan', 'Egypt', 'Antigua and Barbuda', 'Fiji', 'El Salvador', 'Bulgaria', 'Jam